In [3]:
## 돌아가는 거 test하고 나중에 이 파일 전체 삭제하기 !!

In [6]:
# 가상의 테스트 데이터 (DB가 준비되면 실제 DB 연동)
TEST_RESTAURANTS = [
    {"id": 1, "name": "김치찌개 맛집", "category": "한식", "menu": ["김치찌개", "된장찌개"], "open_hours": "09:00-21:00"},
    {"id": 2, "name": "베이징 덕 전문점", "category": "중식", "menu": ["베이징덕", "짜장면"], "open_hours": "10:00-22:00"},
    {"id": 3, "name": "스시야", "category": "일식", "menu": ["초밥", "우동"], "open_hours": "11:00-23:00"},
    {"id": 4, "name": "스테이크 하우스", "category": "양식", "menu": ["스테이크", "파스타"], "open_hours": "12:00-23:00"},
]


In [7]:
def filter_by_category(category):
    """
    사용자가 선택한 카테고리를 기반으로 식당을 필터링하는 함수
    - category: '한식', '중식', '일식' 등 선택된 카테고리
    - "아무거나" 선택 시 모든 식당 반환
    """
    if category == "아무거나":
        return TEST_RESTAURANTS  # 모든 식당 반환
    
    return [restaurant for restaurant in TEST_RESTAURANTS if restaurant["category"] == category]

In [8]:
# 테스트 실행
selected_category = "한식"  # 예시 입력
filtered_restaurants = filter_by_category(selected_category)
print(filtered_restaurants)


[{'id': 1, 'name': '김치찌개 맛집', 'category': '한식', 'menu': ['김치찌개', '된장찌개'], 'open_hours': '09:00-21:00'}]


In [1]:
%pip install openai==0.28

Note: you may need to restart the kernel to use updated packages.


In [1]:
import openai

def regenerate_query(details_input):
    """세부사항을 정리하고 관련 개념을 확장하는 Query 재생성"""
    
    system_prompt = """
    사용자의 검색어를 기반으로 식당 필터링을 위한 Query를 생성하세요.
    
    - 의미를 정리하고 관련 개념을 확장하세요.
    - JSON 형식으로 반환하세요.

    예시:
    - 입력: '조용하고 주차 가능한 곳'
      출력: {'분위기': ['조용함', '아늑한 분위기', '프라이빗 룸'], '주차': ['주차 가능', '발렛 파킹', '근처 공영주차장']}
    - 입력: '단체 모임 하기 좋은 곳'
      출력: {'시설': ['단체석', '넓은 공간', '예약 가능'], '분위기': ['활기찬', '시끌벅적']}
    """

    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": details_input}]
    )

    return response["choices"][0]["message"]["content"]


In [ ]:
# 테스트 실행
details_input = "조용하고 주차 가능한 곳"
expanded_query = regenerate_query(details_input)
print(expanded_query)

{
  "분위기": ["조용함", "아늑한 분위기", "프라이빗 룸"],
  "주차": ["주차 가능", "발렛 파킹", "근처 공영주차장"]
}


In [6]:
def regenerate_query(details_input):
    """
    사용자의 검색어에서 '메뉴명'과 '세부사항(시설, 분위기, 주차)'을 추출하여 JSON 형식으로 변환.
    
    예시:
    - 입력: "김치찌개 조용하고 주차할 수 있는 데서 먹고 싶어"
      출력: {"menu": ["김치찌개"], "분위기": ["조용함"], "주차": ["주차 가능"]}
    - 입력: "초밥집인데 단체석 있는 곳"
      출력: {"menu": ["초밥"], "시설": ["단체석"]}
    """

    system_prompt = """
    사용자의 검색어를 분석하여 JSON 형식으로 반환하세요.
    
    1. 메뉴가 포함되어 있으면 "menu" 키에 해당하는 리스트에 메뉴명을 저장하세요.
    2. 나머지 검색어는 "시설", "분위기", "주차" 등의 키로 분류하여 확장하세요.
    3. JSON 형식으로만 출력하세요.

    예시:
    - 입력: '김치찌개 조용하고 주차할 수 있는 데서 먹고 싶어'
      출력: {"menu": ["김치찌개"], "분위기": ["조용함", "아늑한 분위기"], "주차": ["주차 가능", "발렛 파킹"]}
    - 입력: '초밥집인데 단체석 있는 곳'
      출력: {"menu": ["초밥"], "시설": ["단체석"]}
    """

    try:
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "system", "content": system_prompt},
                      {"role": "user", "content": details_input}]
        )

        # expanded_query = json.loads(response["choices"][0]["message"]["content"])
        expanded_query = response["choices"][0]["message"]["content"]
        return expanded_query

    except Exception as e:
        print("OpenAI API 요청 실패:", e)
        return {}  # 실패 시 빈 딕셔너리 반환

In [8]:
details_test = "김치찌개 조용하고 주차할 수 있는 데서 먹고 싶어"
expanded_query = regenerate_query(details_test)
print(expanded_query)

OpenAI API 요청 실패: name 'json' is not defined
{}


In [ ]:
def filter_by_expanded_query(restaurants, expanded_query):
    """Query 재생성을 기반으로 세부사항과 매칭되는 식당을 필터링"""
    
    matched_restaurants = []

    for res in restaurants:
        res_id, name, facilities, parking, highlights = res["id"], res["name"], res["facilities"], res["parking"], res["highlights"]

        # Query 재구성 결과와 DB 속성을 비교하여 필터링
        matched_details = {
            "식당명": name,
            "편의시설": [f for f in expanded_query.get("시설", []) if f in facilities],
            "주차": [p for p in expanded_query.get("주차", []) if p in parking],
            "이런 점이 좋았어요": [h for h in expanded_query.get("분위기", []) if h in highlights],
        }

        # 하나라도 매칭되는 요소가 있으면 추가
        if any(matched_details.values()):
            matched_restaurants.append(matched_details)

    return matched_restaurants



In [ ]:
# 필터링 테스트 실행
filtered_results = filter_by_expanded_query(TEST_RESTAURANTS, expanded_query)
print(filtered_results)

NameError: name 'TEST_RESTAURANTS' is not defined